In [58]:
%reload_ext autoreload
%autoreload 2
import tensorflow as tf
import sys
import math
import numpy as np
sys.path.append('../pytools')
import d2l

d2l.gpu_mem_init()

Memory growth enabled for PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'): True


In [59]:
net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(10)
])
x = tf.random.uniform((2, 20, 2))
net(x)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[ 0.24339932, -0.32407302,  0.03625934, -0.02166909,  0.35151708,
         0.4057223 ,  0.215098  , -0.03500882,  0.22022352,  0.2466392 ],
       [ 0.23594415, -0.40380743,  0.04983503,  0.04183488,  0.13467623,
         0.20341524,  0.26040128, -0.20052058,  0.35503572,  0.30703652]],
      dtype=float32)>

In [60]:
class MLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.hidden = tf.keras.layers.Dense(256, activation=tf.nn.relu)
        self.out = tf.keras.layers.Dense(10)
    def call(self, x):
        return self.out(self.hidden(x))

In [61]:
net = MLP()
x = tf.random.uniform((2, 16))
net(x)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[ 0.14893168, -0.2283766 , -0.05515024, -0.04604724, -0.10981039,
        -0.16195588, -0.10656617,  0.05857914, -0.05026571,  0.17040884],
       [ 0.02062778, -0.25661013,  0.14620763, -0.12517123, -0.18969613,
        -0.25388798, -0.17432716,  0.07707973, -0.11335789, -0.03260124]],
      dtype=float32)>

In [62]:
class MySequential(tf.keras.Model):
    def __init__(self, *args):
        super().__init__()
        self.models = []
        for blk in args:
            self.models.append(blk)
    def call(self, x):
        for blk in self.models:
            x = blk(x)
        return x

In [63]:
net = MySequential(
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(10)
)
net(x)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[-0.23828478,  0.34212494, -0.1052343 ,  0.06910454,  0.14488065,
        -0.6529054 , -0.06468116, -0.1710736 , -0.04827119,  0.05332965],
       [-0.18628061,  0.35755607, -0.10742956,  0.02469761,  0.09701915,
        -0.4673486 , -0.09893426, -0.11672848,  0.12406464,  0.25873494]],
      dtype=float32)>

In [64]:
def cond(x,y):
    return tf.reduce_sum(tf.abs(x)) > 1
def body(x,y):
    out = y / 2
    return out,out
class FixedHiddenMlp(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()
        self.random_weight = tf.random.uniform((20,20))
        self.dense = tf.keras.layers.Dense(20, activation='relu')
    def call(self, x):
        x = self.flatten(x)
        x = tf.nn.relu(tf.matmul(x, self.random_weight) + 1)
        x = self.dense(x)
        #原书中这个地方直接用python表达式，直接调用FixedHiddenMlp可以跑成功；但是在下文中作为tf.keras.Sequential的某一层
        #运行会失败，改成tf.while_loop能运行成功
        out = tf.while_loop(cond, body, [x,x])
        return tf.reduce_sum(out)

In [65]:
net = FixedHiddenMlp()
x = tf.random.uniform((2,20))
net(x)

<tf.Tensor: shape=(), dtype=float32, numpy=1.0914294719696045>

In [66]:
class NestMLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.net = tf.keras.Sequential()
        self.net.add(tf.keras.layers.Dense(64, activation='relu'))
        self.net.add(tf.keras.layers.Dense(32, activation='relu'))
        self.dense = tf.keras.layers.Dense(16, activation='relu')
    def call(self, x):
        return self.dense(self.net(x))

In [67]:
mixnet = tf.keras.Sequential()
mixnet.add(NestMLP())
mixnet.add(tf.keras.layers.Dense(20))
mixnet.add(FixedHiddenMlp())

In [68]:
x = tf.random.uniform((2, 20))
mixnet(x)

<tf.Tensor: shape=(), dtype=float32, numpy=1.5485560894012451>